# Evolutionary Algorithms Lab - Set Covering Problem

In [220]:
import logging
import numpy as np
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from matplotlib import cm
import random

logging.basicConfig(
    format="[%(asctime)s] %(levelname)s: %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

N_VALUES = [5, 10, 20, 100, 500, 1000]
SEED = 42

def problem(N, seed=SEED):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [221]:


from torch import rand


def problem(N, seed=SEED):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

def goal_test(N, solution):
    set_sol = set([item for sublist in solution for item in sublist])
    goal = set(range(N))
    return set_sol == goal

def fitness_eval(solutions):
    weight = []
    for sol in solutions:
        w = sum(len(_) for _ in sol)
        weight.append(w)

    return weight

#the issue is here somewhere over the rainbow
def offspring_gen(parent, all_lists, λ, σ, N):

    offspring = [parent]
    for l in range(λ - 1):
        new_offspring = parent
        safety_counter = 0
        mutation_factor = 1 + round(abs(np.random.normal(0, σ)))

        for mut in range(mutation_factor):
            new_offspring.pop(random.randint(0, abs(len(new_offspring)-1)))
            random_list = random.choice(all_lists)
            while not goal_test(N, new_offspring): 
                if random_list not in new_offspring and random_list not in parent:
                    new_offspring.append(random_list)
                else:
                    safety_counter+=1
                    random_list = random.choice(all_lists)
     
                if not safety_counter < len(all_lists):
                    new_offspring = parent
                    continue

        offspring.append(new_offspring)
    
    return offspring


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [222]:
def plus_lambda_ea(population, all_lists, λ, σ, n):
    population = offspring_gen(population, all_lists, λ, σ, n)
    
    for n in range(10 // λ):

        fitness = fitness_eval(population)
        parent = population[np.argmin(fitness)]
        population = offspring_gen(parent, all_lists, λ, σ, n)
    
    fitness = fitness_eval(population)
    final_sol = population[np.argmin(fitness)]

    return final_sol

In [223]:
# 1 + λ ES

λ = 5
σ = 0.05

for n in N_VALUES:
    all_lists = problem(n, SEED)
    random_list = random.choice(all_lists)
    start_element = []

    while not goal_test(n, start_element):
        if random_list not in start_element:
            start_element.append(random_list)
        random_list = random.choice(all_lists)
    if n < 20:    
        logging.info(f"start element {start_element}")
    sol = plus_lambda_ea(start_element, all_lists, λ, σ, n)

    logging.info(f"N = {n}, best solution: w={sum(len(_) for _ in sol)} (bloat={(sum(len(_) for _ in sol)-n)/n*100:.0f}%)")

[23:50:04] INFO: start element [[1], [2], [4], [0, 1], [0], [2, 4], [3]]
